In [1]:
from pathlib import *
import numpy as np
import pandas as pd
import mne
import six, os
np.set_printoptions(linewidth=999999, precision=6, suppress=True, threshold=99999)
pd.set_option('display.width', 999999)
pd.set_option('display.max_colwidth', 300)
import nibabel as nib
import matplotlib.pyplot as plt
#plt.interactive(False)
from pylabs.correlation.atlas import mori_region_labels, JHUtracts_region_labels
from pylabs.projects.bbc.pairing import foster_behav_data, control_behav_data, behav_list
from pylabs.projects.bbc.correl.atlases import atlases
from pylabs.utils import run_subprocess, WorkingContext
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())
project = 'bbc'
statsdir = fs/project/'stats'/'py_correl_5thpass_cthr15_n5000'    #'py_correl_3rdpass'   #'py_correl_5thpass'
fname_templ = '{pool}_{mod}.nii'
cluster_idx_fname_templ = '{pool}_{mod}_{behav}_t{dir}_cluster_index.nii.gz'

In [18]:
# prepare cluster roi data
cluster_fname = 'cluster_report.csv'
clusters = pd.DataFrame.from_csv(str(statsdir/cluster_fname))
clusters.rename(index=str, columns={' k': 'k', ' x': 'x', ' y': 'y', ' z': 'z', ' name': 'name', ' mori': 'mori', ' JHU_tracts': 'JHU_tracts'}, inplace=True)
clusters.drop(clusters[(clusters.mori == 'Background') & (clusters.JHU_tracts == 'Background')].index, inplace=True)
clusters['mod'] = clusters['name'].str.split('-').apply(lambda x: x[-1])
clusters['pool'] = clusters['name'].str.split('-').apply(lambda x: x[-2])
clusters['dir'] = clusters['name'].str.split('-').apply(lambda x: x[-3])
clusters['behav'] = clusters['name'].str.split('-').apply(lambda x: x[:-3]).str.join('-')
clusters['clu_idx_fname'] = clusters[['pool', 'mod', 'behav', 'dir']].apply(lambda x : cluster_idx_fname_templ.format(pool=x[0], mod=x[1], behav=x[2], dir=x[3]), axis=1)
with WorkingContext(str(statsdir)):
    try:
        for index, row in clusters.iterrows():
            print os.getcwd(), row['clu_idx_fname']
            print index, row
            print Path(row['clu_idx_fname']).is_file()
            clusters.set_value(str(index), 'idx_data', nib.load(row['clu_idx_fname']).get_data())
            clusters.set_value(str(index), 'idx_zooms', nib.load(row['clu_idx_fname']).header.get_zooms())
            clusters.set_value(str(index), 'idx_shape', nib.load(row['clu_idx_fname']).get_data().shape)
    except:
        print('an error has occured loading data into dataframe.')
clusters.head(10)

/mnt/users/js/bbc/stats/py_correl_5thpass_cthr15_n5000 foster_FA_PPVTSS_tpos_cluster_index.nii.gz
201 k                                                        16
x                                                        76
y                                                        41
z                                                        35
name                                   PPVTSS-pos-foster-FA
mori                                             CEREBELLUM
JHU_tracts                                     Cerebellum_L
mod                                                      FA
pool                                                 foster
dir                                                     pos
behav                                                PPVTSS
clu_idx_fname    foster_FA_PPVTSS_tpos_cluster_index.nii.gz
Name: 201, dtype: object
True
an error has occured loading data into dataframe.


,k,x,y,z,name,mori,JHU_tracts,mod,pool,dir,behav,clu_idx_fname
cluster-index,,,,,,,,,,,,
201,16,76,41,35,PPVTSS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,PPVTSS,foster_FA_PPVTSS_tpos_cluster_index.nii.gz
208,21,74,38,36,CTOPPphoaCS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,CTOPPphoaCS,foster_FA_CTOPPphoaCS_tpos_cluster_index.nii.gz
313,23,76,50,57,PPVTSS-pos-foster-MD,Fornix(cres)_Stria_terminalis_left,Inferior_fronto_occipital_fasciculus_L,MD,foster,pos,PPVTSS,foster_MD_PPVTSS_tpos_cluster_index.nii.gz
279,21,63,38,81,CTOPPphoaCS-pos-foster-MD,Substancia_Nigra_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
256,19,58,80,64,CTOPPphoaCS-pos-foster-MD,CAUDATE_NUCLEUS_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
229,18,54,80,60,CTOPPphoaCS-pos-foster-MD,CAUDATE_NUCLEUS_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
239,21,63,38,81,CTOPPphoaCS-pos-foster-RD,Substancia_Nigra_left,Background,RD,foster,pos,CTOPPphoaCS,foster_RD_CTOPPphoaCS_tpos_cluster_index.nii.gz
315,37,76,48,58,PPVTSS-pos-foster-AD,Cingulum_(hippocampus)_left,Superior_longitudinal_fasciculus_L,AD,foster,pos,PPVTSS,foster_AD_PPVTSS_tpos_cluster_index.nii.gz
240,34,56,70,53,PPVTSS-pos-foster-AD,Cerebral_peduncle_left,Anterior_thalamic_radiation_L,AD,foster,pos,PPVTSS,foster_AD_PPVTSS_tpos_cluster_index.nii.gz


In [6]:
# prepare data for correlations
data = pd.DataFrame(index=list(set(clusters['pool'].values)), columns=list(set(clusters['mod'].values)))
with WorkingContext(str(statsdir)):
    try:
        for pool in list(set(clusters['pool'].values)):
            for mod in list(set(clusters['mod'].values)):
                fname = fname_templ.format(pool=pool, mod=mod)
                data.set_value(pool, mod, nib.load(fname).get_data())
    except:
        print('an error has occured loading data into dataframe.')

In [15]:
clusters.set_value('201', 'idx_data', 1.0)

,k,x,y,z,name,mori,JHU_tracts,mod,pool,dir,behav,clu_idx_fname,idx_data
cluster-index,,,,,,,,,,,,,
201,16,76,41,35,PPVTSS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,PPVTSS,foster_FA_PPVTSS_tpos_cluster_index.nii.gz,1.0
208,21,74,38,36,CTOPPphoaCS-pos-foster-FA,CEREBELLUM,Cerebellum_L,FA,foster,pos,CTOPPphoaCS,foster_FA_CTOPPphoaCS_tpos_cluster_index.nii.gz,NaN
313,23,76,50,57,PPVTSS-pos-foster-MD,Fornix(cres)_Stria_terminalis_left,Inferior_fronto_occipital_fasciculus_L,MD,foster,pos,PPVTSS,foster_MD_PPVTSS_tpos_cluster_index.nii.gz,NaN
279,21,63,38,81,CTOPPphoaCS-pos-foster-MD,Substancia_Nigra_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz,NaN
256,19,58,80,64,CTOPPphoaCS-pos-foster-MD,CAUDATE_NUCLEUS_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz,NaN
229,18,54,80,60,CTOPPphoaCS-pos-foster-MD,CAUDATE_NUCLEUS_left,Background,MD,foster,pos,CTOPPphoaCS,foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz,NaN
239,21,63,38,81,CTOPPphoaCS-pos-foster-RD,Substancia_Nigra_left,Background,RD,foster,pos,CTOPPphoaCS,foster_RD_CTOPPphoaCS_tpos_cluster_index.nii.gz,NaN
315,37,76,48,58,PPVTSS-pos-foster-AD,Cingulum_(hippocampus)_left,Superior_longitudinal_fasciculus_L,AD,foster,pos,PPVTSS,foster_AD_PPVTSS_tpos_cluster_index.nii.gz,NaN
240,34,56,70,53,PPVTSS-pos-foster-AD,Cerebral_peduncle_left,Anterior_thalamic_radiation_L,AD,foster,pos,PPVTSS,foster_AD_PPVTSS_tpos_cluster_index.nii.gz,NaN


In [9]:
for index, row in clusters.iterrows():
    print index
    print row['clu_idx_fname']

201
foster_FA_PPVTSS_tpos_cluster_index.nii.gz
208
foster_FA_CTOPPphoaCS_tpos_cluster_index.nii.gz
313
foster_MD_PPVTSS_tpos_cluster_index.nii.gz
279
foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
256
foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
229
foster_MD_CTOPPphoaCS_tpos_cluster_index.nii.gz
239
foster_RD_CTOPPphoaCS_tpos_cluster_index.nii.gz
315
foster_AD_PPVTSS_tpos_cluster_index.nii.gz
240
foster_AD_PPVTSS_tpos_cluster_index.nii.gz
121
foster_AD_PPVTSS_tpos_cluster_index.nii.gz
282
foster_AD_CTOPPphoaCS_tpos_cluster_index.nii.gz
246
foster_AD_CTOPPphomCS_tpos_cluster_index.nii.gz
116
foster_GM_PPVTSS_tpos_cluster_index.nii.gz
109
foster_GM_PPVTSS_tpos_cluster_index.nii.gz
100
foster_GM_PPVTSS_tpos_cluster_index.nii.gz
121
foster_GM_PPVTSS_tpos_cluster_index.nii.gz
92
foster_GM_PPVTSS_tpos_cluster_index.nii.gz
101
foster_GM_CTOPPphoaCS_tpos_cluster_index.nii.gz
102
foster_GM_CTOPPphoaCS_tpos_cluster_index.nii.gz
116
foster_GM_PATrhyTotSS_tpos_cluster_index.nii.gz
100
foster_